In [11]:
import plotly.graph_objects as go
import plotly.express as px

import numpy as np
import pandas as pd
import os
import json

In [2]:
data_directory = "../data/ontario/"

individual_prefix = "table_person_ontario_"
total_prefix = "total_ontario_"

In [27]:
files = os.listdir(data_directory)

people_files = [f for f in files if individual_prefix in f]
total_files = [f for f in files if total_prefix in f]

In [57]:
peoples = {}

#TODO
#how to get dates so I can have transmission type over time
for file in people_files:
    path = data_directory + file
    with open(path, 'r') as outfile:
        for l in outfile : 
            x = json.loads(l)
            
            if x['case number'] in peoples :
                current = peoples[x['case number']]
                person = x['patient']
                if len(person.split(" ")) >1 :
                    
                    age = int(person.split(" ")[0][:-1])
                    gender = person.split(" ")[1]
                else:
                    age = person.split(" ")[0]
                    gender = person.split(" ")[0]
                place = x['public health unit']
                transmission = x['transmission']
                status = x['status']
                new = {"age" : age, 
                        'gender' : gender, 
                        'public health unit' : place,
                        'transmission' : transmission,
                        'status' : status
                                            }
                #check if info has been updated
                for key, item in new.items():
                    if key == 'case number':
                        continue
                    if current[key] == 'pending':
                        current[key] = item
                    
                peoples[x['case number']] = current
            else :
                person = x['patient']
                
                if len(person.split(" ")) >1 :
                    
                    age = int(person.split(" ")[0][:-1])
                    gender = person.split(" ")[1]
                else:
                    age = person.split(" ")[0]
                    gender = person.split(" ")[0]
                place = x['public health unit']
                transmission = x['transmission']
                status = x['status']
                peoples[x['case number']] = {"age" : age, 
                                            'gender' : gender, 
                                            'public health unit' : place,
                                            'transmission' : transmission,
                                            'status' : status

                                            }

In [64]:
peoples_list = []
for key, people in peoples.items():
    people["case number"] = key
    peoples_list.append(people)

df = pd.DataFrame.from_dict(peoples_list)
male = [ {"age" :p["age"], "zone" : p["public health unit"]} for p in peoples_list if p["gender"] == "Male" and p["age"] != 'pending' ]
male = pd.DataFrame.from_dict(male)
          
female = [ {"age" :p["age"], "zone" : p["public health unit"]} for p in peoples_list if p["gender"] == "Female" and p["age"] != 'pending']
female = pd.DataFrame.from_dict(female)

#transmission = [ {"transmission" :p["age"], "zone" : p["public health unit"]} for p in peoples_list if p["gender"] == "female"]



In [77]:
fig = px.histogram(male, x="age", color = "zone", color_discrete_sequence= px.colors.qualitative.Light24)
fig.update_layout(
    title_text='Male Infected by Age', # title of plot
    xaxis_title_text='Age', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.7, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [80]:
fig = px.histogram(female, x="age", color = "zone", color_discrete_sequence= px.colors.qualitative.Light24)
fig.update_layout(
    title_text='Female Infected by Age', # title of plot
    xaxis_title_text='Age', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.7, # gap between bars of adjacent location coordinates
    #bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [44]:
fig = px.histogram(full_df, x= "transmission")
fig.show()